# **양자과학기술 기관명 통일**

- suborganization과 city를 기준으로 organization 통일
- suborganization이 NaN인 경우 organization을 suborganization으로 대체
- 주소가 같으면 짧은 organization 이름으로 통일 
- duplicated row 제거

In [1]:
import pandas as pd 
import numpy as np

# dir = 'I:/Data_for_practice/Rfiles/QuanTech/'
dir = 'D:/GD_awekimm/[YU]/[Project]/[Quantum]/Quantum_2nd/04_Analysis/QuanTech_R2/R file/'

## New Version

In [7]:
df = pd.read_csv(dir+'quant_inst_ed_eu_val.csv')

df.replace('NULL', np.nan, inplace=True)

df_for_mapping = df.dropna(subset=['full_address']).copy()

address_to_org = {}
address_to_suborg = {}

grouped_by_address = df_for_mapping.groupby('full_address')

for address, group in grouped_by_address:
    # Organization
    # Get unique, non-null organization names for the address group
    orgs = group['organization'].dropna().unique()
    if len(orgs) > 0:
        # Select the longest string as the canonical name
        canonical_org = max(orgs, key=len)
        address_to_org[address] = canonical_org

    # Suborganization
    # Get unique, non-null suborganization names for the address group
    suborgs = group['suborganization'].dropna().unique()
    if len(suborgs) > 0:
        # Select the longest string as the canonical name
        canonical_suborg = max(suborgs, key=len)
        address_to_suborg[address] = canonical_suborg

df['organization_clean'] = df['full_address'].map(address_to_org)
df['suborganization_clean'] = df['full_address'].map(address_to_suborg)

# df[['pubid','organization_clean','suborganization_clean']].drop_duplicates().to_csv(dir+'quant_inst_ed_eu_val_list.csv', index=False)
df[['pubid','organization_clean','suborganization_clean']].drop_duplicates().to_csv(dir+'quant_inst_ed_eu_val_list.csv', index=False)
df.to_csv(dir+'quant_inst_ed_eu_val_clean.csv', index=False)
print(df.organization.nunique())
print(df.suborganization.nunique())
print(df.organization_clean.nunique())
print(df.suborganization_clean.nunique())
df.head()

21506
33441
12683
27670


,pubid,qc_category,addr_num,SEQ_NO,institution_id_te,organization,suborganization,full_address,city,state,country,zip,pubyear,organization_clean,suborganization_clean
0,52091.0,qc111,1,1.0,82651833,UNIVERSITY OF AQUILA,DIPARTIMENTO ENERGET,"UNIV AQUILA, DIPARTIMENTO ENERGET, I-67040 LAQ...",LAQUILA,NaN,ITALY,I-67040,1998,UNIVERSITY OF AQUILA,DIPARTIMENTO ENERGET
1,52091.0,qc111,2,NaN,13505120,CNR,IST ELETTR STATO SOLIDO,"CNR, IST ELETTR STATO SOLIDO, I-00156 ROME, ITALY",ROME,NaN,ITALY,I-00156,1998,CONSIGLIO NAZIONALE DELLE RICERCHE (CNR),IST ELETTR STATO SOLIDO
2,52091.0,qc111,2,NaN,15468272,CONSIGLIO NAZIONALE DELLE RICERCHE (CNR),IST ELETTR STATO SOLIDO,"CNR, IST ELETTR STATO SOLIDO, I-00156 ROME, ITALY",ROME,NaN,ITALY,I-00156,1998,CONSIGLIO NAZIONALE DELLE RICERCHE (CNR),IST ELETTR STATO SOLIDO
3,52091.0,qc111,1,1.0,67923178,UNIV AQUILA,DIPARTIMENTO ENERGET,"UNIV AQUILA, DIPARTIMENTO ENERGET, I-67040 LAQ...",LAQUILA,NaN,ITALY,I-67040,1998,UNIVERSITY OF AQUILA,DIPARTIMENTO ENERGET
4,62208.0,qc111,2,NaN,39506327,KYUSHU UNIV,DEPT ELECT DEVICE ENGN,"KYUSHU UNIV, DEPT ELECT DEVICE ENGN, FUKUOKA 8...",FUKUOKA,NaN,JAPAN,812,1998,KYUSHU UNIVERSITY,DEPT ELECT DEVICE ENGN


In [8]:
org_mapping = {
    "UNIV AQUILA": "UNIVERSITY OF AQUILA",
    "CNR": "CONSIGLIO NAZIONALE DELLE RICERCHE (CNR)",
    "CONSIGLIO NAZIONALE DELLE RICERCHE": "CONSIGLIO NAZIONALE DELLE RICERCHE (CNR)",
    "KYUSHU UNIV": "KYUSHU UNIVERSITY",
    "TECHNICAL UNIVERSITY OF BERLIN": "PHYSIKALISCH-TECHNISCHE BUNDESANSTALT (PTB)",
    "INST BERLIN": "PHYSIKALISCH-TECHNISCHE BUNDESANSTALT (PTB)",
    "UNIV LANCASTER": "LANCASTER UNIVERSITY",
    "NATIONAL ACADEMY OF SCIENCES UKRAINE": "LASHKARYOV INSTITUTE OF SEMICONDUCTOR PHYSICS",
    "NATL ACAD SCI UKRAINE": "LASHKARYOV INSTITUTE OF SEMICONDUCTOR PHYSICS",
    "UNIV JENA": "FRIEDRICH SCHILLER UNIVERSITY OF JENA",
    "COMMUNAUTE UNIVERSITE GRENOBLE ALPES": "UNIVERSITE GRENOBLE ALPES (UGA)",
    "UNIVERSITE GRENOBLE ALPES": "UNIVERSITE GRENOBLE ALPES (UGA)",
    "CNRS": "CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE (CNRS)",
    "CEA": "CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE (CNRS)",
    "LETI": "CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE (CNRS)",
    "MINATEC": "CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE (CNRS)",
    "INSTITUT NATIONAL POLYTECHNIQUE DE GRENOBLE": "UNIVERSITE GRENOBLE ALPES (UGA)",
    "UNIVERSITY OF NEW SOUTH WALES SYDNEY": "UNIVERSITY OF NEW SOUTH WALES",
    "ECOLE POLYTECH FED LAUSANNE": "ECOLE POLYTECHNIQUE FEDERALE DE LAUSANNE",
    "QUANTUM MOT TECHNOL": "QUANTUM MOTION TECHNOLOGIES",
    "IBM": "IBM RESEARCH ZURICH",
    "UNIV PARIS 11": "UNIVERSITE PARIS SUD",
    "UNIV PARIS SUD 11": "UNIVERSITE PARIS SUD",
    "UNIV CAMBRIDGE": "UNIVERSITY OF CAMBRIDGE",
    "UNIV JYVASKYLA": "UNIVERSITY OF JYVASKYLA",
    "UNIV GLASGOW": "UNIVERSITY OF GLASGOW",
    "UNIV BASEL": "UNIVERSITY OF BASEL",
    "UNIV COPENHAGEN": "NIELS BOHR INSTITUTE",
    "NIELS BOHR INST": "NIELS BOHR INSTITUTE",
    "UNIVERSITY OF COPENHAGEN": "NIELS BOHR INSTITUTE",
    "UNIV PARIS 06": "SORBONNE UNIVERSITE",
    "UPMC": "SORBONNE UNIVERSITE",
    "PIERRE & MARIE CURIE UNIV": "SORBONNE UNIVERSITE",
    "SORBONNE UNIV": "SORBONNE UNIVERSITE",
    "UNIV PIERRE & MARIE CURIE": "SORBONNE UNIVERSITE",
    "UNIV P & M CURIE": "SORBONNE UNIVERSITE",
    "JOHANNES GUTENBERG UNIV MAINZ": "JOHANNES GUTENBERG UNIVERSITY MAINZ",
    "UNIV KONSTANZ": "UNIVERSITY OF KONSTANZ",
    "UNIV TECHNOL SYDNEY": "UNIVERSITY OF TECHNOLOGY SYDNEY",
    "KARLSRUHE INST TECHNOL": "KARLSRUHE INSTITUTE OF TECHNOLOGY",
    "KIT": "KARLSRUHE INSTITUTE OF TECHNOLOGY",
    "UNIV LINZ": "JOHANNES KEPLER UNIVERSITY LINZ",
    "JOHANNES KEPLER UNIV LINZ": "JOHANNES KEPLER UNIVERSITY LINZ",
    "UNIV TWENTE": "UNIVERSITY OF TWENTE",
    "UNIV AUTONOMA MADRID": "UNIVERSIDAD AUTONOMA DE MADRID",
    "UNIV LEEDS": "UNIVERSITY OF LEEDS",
    "UNIV SHEFFIELD": "UNIVERSITY OF SHEFFIELD",
    "UNIV DELFT TECHNOL": "DELFT UNIVERSITY OF TECHNOLOGY",
    "TU DELFT": "DELFT UNIVERSITY OF TECHNOLOGY",
    "UNIV GHENT": "GHENT UNIVERSITY",
    "UNIV SAARLAND": "SAARLAND UNIVERSITY",
    "UNIV GENEVA": "UNIVERSITY OF GENEVA",
    "UNIV AUTONOMA BARCELONA": "UNIVERSITAT AUTONOMA DE BARCELONA",
    "UNIV TURKU": "UNIVERSITY OF TURKU",
    "UNIV OXFORD": "UNIVERSITY OF OXFORD",
    "UNIV WATERLOO": "UNIVERSITY OF WATERLOO",
    "UNIV ILLINOIS": "UNIVERSITY OF ILLINOIS URBana champaign",
    "UNIV ILLINOIS URBANA CHAMPAIGN": "UNIVERSITY OF ILLINOIS URBANA CHAMPAIGN",
    "UNIV COMPLUTENSE MADRID": "COMPLUTENSE UNIVERSITY OF MADRID",
    "CSIC": "AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS",
    "UNIV ZARAGOZA": "AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS",
    "AALTO UNIV": "AALTO UNIVERSITY",
    "UNIV HELSINKI": "AALTO UNIVERSITY",
    "HELSINKI UNIV TECHNOL": "AALTO UNIVERSITY",
    "UNIV VIENNA": "UNIVERSITY OF VIENNA",
    "UNIV WISCONSIN MADISON": "UNIVERSITY OF WISCONSIN MADISON",
    "UNIV WISCONSIN": "UNIVERSITY OF WISCONSIN MADISON",
    "RWTH AACHEN UNIV": "RWTH AACHEN UNIVERSITY",
    "AACHEN UNIV": "RWTH AACHEN UNIVERSITY",
    "UNIV BARCELONA": "UNIVERSITY OF BARCELONA"
}

suborg_mapping = {
    "DEPT PHYS": "FAC MATH & PHYS",
    "CLARENDON LAB": "RUDOLF PEIERLS CTR THEORET PHYS",
    "DEPT ELECT & COMP ENGN": "DEPT PHYS & ASTRON",
    "INST QUANTUM COMPUTING": "DEPT PHYS & ASTRON",
    "WATERLOO INST NANOTECHNOL": "DEPT PHYS & ASTRON",
    "FREDERICK SEITZ MAT RES LAB": "MICRO & NANOTECHNOL LAB",
    "CTR MAT RES": "FREDERICK SEITZ MAT RES LAB",
    "DEPT PHYS CHEM": "FAC CHEM",
    "DEPT INORGAN CHEM": "FAC CHEM",
    "DEPT ORGAN CHEM": "FAC CHEM",
    "DEPT APPL PHYS": "FAC CHEM",
    "MATSENS": "FAC CHEM",
    "ZARAGOZA INST NANOSCI & MAT ADV ARAGON": "FAC CHEM",
    "CSIC": "FAC CHEM",
    "INMA": "FAC CHEM",
    "UNIV ZARAGOZA": "FAC CHEM",
    "INST MAT SCI ARAGON": "FAC CHEM",
    "DEPT CONDENSED MATTER PHYS": "FAC CHEM",
    "ICMA": "FAC CHEM",
    "DEPT ENGN & MECH": "LOW TEMP LAB",
    "HELSINKI UNIV TECHNOL": "LOW TEMP LAB",
    "CAVENDISH LAB": "DEPT PHYS",
    "DEPT ENGN": "DEPT PHYS",
    "NANOSCI CTR": "DEPT PHYS",
    "DEPT MAT SCI & METALLURGY": "DEPT PHYS",
    "HITACHI CAMBRIDGE LAB": "DEPT PHYS",
    "DEPT PHYS & ASTRONOMY": "DEPT PHYS",
    "DEPT CHEM": "DEPT PHYS",
    "DEPT APPL MATH & THEORET PHYS": "DEPT PHYS",
    "CTR NANOSCI": "DEPT PHYS",
    "LONDON CTR NANOTECHNOL": "DEPT PHYS & ASTRONOMY",
    "PHYS DEPT": "DEPT PHYS",
    "SCH PHYS": "CTR FOR QUANTUM COMPUTING & COMMUNICATION TECHNOLOGY",
    "DEPT PHYS & CTR NANOSCI": "DEPT PHYS",
    "QUTE CTR": "DEPT PHYS",
    "CTR APPL PHOTON": "DEPT PHYS",
    "CTR FOR INTEGRATED QUANTUM SCIENCE & TECHNOLOGY": "DEPT PHYS",
    "CTR FOR APPLIED QUANTUM TECHNOLOGY": "DEPT PHYS",
    "CTR FOR QUANTUM INFORMATION": "DEPT PHYS",
    "CTR FOR QUANTUM PHYSICS": "DEPT PHYS",
    "CTR FOR QUANTUM SCIENCE & ENGINEERING": "DEPT PHYS",
    "CTR FOR SOLID STATE PHYSICS & NEW MATERIALS": "DEPT PHYS",
    "CTR FOR SPINTRONICS & QUANTUM INFORMATION": "DEPT PHYS",
    "CTR FOR THEORETICAL & COMPUTATIONAL PHYSICS": "DEPT PHYS"
}

In [10]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict

def create_comprehensive_cleaning_maps(dataframe):
    """
    데이터프레임을 입력받아, Pair Matrix와 주소 정보로 
    가장 포괄적인 Organization 및 Suborganization 정제용 딕셔너리를 생성
    """
    print("ANALYSIS START:")
    
    organization_map = {}
    suborganization_map = {}
    
    df = dataframe.copy()
    df.replace('NULL', np.nan, inplace=True)
    df.fillna({'organization': '', 'suborganization': ''}, inplace=True)

    def generate_advanced_key(text):
        text = str(text).lower()
        
        text = re.sub(r'\s*\([^)]*\)', '', text)
        
        abbreviations = {
            'univ': 'university', 'inst': 'institute', 'dept': 'department', 'ctr': 'center',
            'lab': 'laboratory', 'sch': 'school', 'engn': 'engineering', 'phys': 'physics',
            'chem': 'chemistry', 'sci': 'science', 'tech': 'technology', 'technol': 'technology',
            'natl': 'national', 'fed': 'federal', 'res': 'research', 'adv': 'advanced',
            'mat': 'materials', 'syst': 'systems', 'elect': 'electrical', 'int': 'international',
            'polytech': 'polytechnique', 'metrol': 'metrology', 'nucl': 'nuclear',
            'acad': 'academy', 'coll': 'college', '&': 'and'
        }
        for abbr, full in abbreviations.items():
            text = re.sub(r'\b' + re.escape(abbr) + r'\b', full, text)
        
        text = re.sub(r'[^\w\s]', ' ', text) 
        # text = re.sub(r'\b\d+\b', '', text) # Remove numbers 
                
        stop_words = {'of', 'the', 'for', 'and', 'a', 'an', 'in', 'on', 'at', 'de', 'la', 'di'}
        words = sorted([word for word in text.split() if word not in stop_words])
        
        return ' '.join(words)

    print(" -> Step 1: Pair Matrix comparison analysis")
    
    df['affiliation_pair'] = df['organization_clean'].str.strip() + '|' + df['suborganization_clean'].str.strip()
    unique_pairs = df['affiliation_pair'].unique()

    grouped_by_key = defaultdict(list)
    for pair in unique_pairs:
        key = generate_advanced_key(pair)
        if key: 
            grouped_by_key[key].append(pair)

    for key, group in grouped_by_key.items():
        if len(group) > 1:
            # Choose longest one
            canonical_pair = max(group, key=lambda x: len(x.replace('|','')))
            for variant_pair in group:
                if variant_pair != canonical_pair:
                    variant_org, variant_suborg = variant_pair.split('|', 1)
                    canonical_org, canonical_suborg = canonical_pair.split('|', 1)
                    
                    if variant_org != canonical_org and variant_org:
                        organization_map[variant_org] = canonical_org
                    if variant_suborg != canonical_suborg and variant_suborg:
                        suborganization_map[variant_suborg] = canonical_suborg

    print(f" -> Step 1: Pair Matrix analysis {len(organization_map) + len(suborganization_map)} created.")

    # Full_address based analysis
    if 'full_address' in df.columns:
        print(" Step 2: full_address based analysis")
        df_with_address = df.dropna(subset=['full_address']).copy()
        grouped_by_address = df_with_address.groupby('full_address')

        for address, group in grouped_by_address:
            # Organization
            unique_orgs = [org for org in group['organization_clean'].unique() if org]
            if len(unique_orgs) > 1:
                canonical_org = max(unique_orgs, key=len)
                for org in unique_orgs:
                    if org != canonical_org and org not in organization_map:
                         organization_map[org] = canonical_org
            # Suborganization
            unique_suborgs = [suborg for suborg in group['suborganization_clean'].unique() if suborg]
            if len(unique_suborgs) > 1:
                canonical_suborg = max(unique_suborgs, key=len)
                for suborg in unique_suborgs:
                    if suborg != canonical_suborg and suborg not in suborganization_map:
                        suborganization_map[suborg] = canonical_suborg
    
    print(f"ANALYSIS COMPLETE: Total {len(organization_map) + len(suborganization_map)} created.")
    return organization_map, suborganization_map

try:
    df_original = pd.read_csv(dir+'quant_inst_ed_eu_val_clean.csv')

    org_map, suborg_map = create_comprehensive_cleaning_maps(df_original)

    print("\nCleaning Process: ")
    df_cleaned = df_original.copy()

    df_cleaned['organization_cleaned'] = df_cleaned['organization_clean']
    df_cleaned['suborganization_cleaned'] = df_cleaned['suborganization_clean']

    df_cleaned['organization_cleaned'] = df_cleaned['organization_cleaned'].replace(org_map)
    df_cleaned['suborganization_cleaned'] = df_cleaned['suborganization_cleaned'].replace(suborg_map)

    df_cleaned.to_csv(dir+'quant_inst_ed_eu_val_cleaned.csv', 
                      index=False, encoding='utf-8-sig')

    print(f"\n Completed.!")
    
    original_org_count = df_original['organization_clean'].nunique()
    cleaned_org_count = df_cleaned['organization_cleaned'].nunique()
    original_suborg_count = df_original['suborganization_clean'].nunique()
    cleaned_suborg_count = df_cleaned['suborganization_cleaned'].nunique()
    
    print("\n--- Comparison ---")
    print(f"Organization:    {original_org_count}  ->  {cleaned_org_count}  (Reduced: {original_org_count - cleaned_org_count}개)")
    print(f"Suborganization: {original_suborg_count}  ->  {cleaned_suborg_count}  (Reduced: {original_suborg_count - cleaned_suborg_count}개)")

except FileNotFoundError:
    print(f"ERROR: input_file Error.")
except Exception as e:
    print(f"ERROR: Unexpected Error: {e}")

ANALYSIS START:
 -> Step 1: Pair Matrix comparison analysis
 -> Step 1: Pair Matrix analysis 1363 created.
 Step 2: full_address based analysis
ANALYSIS COMPLETE: Total 1363 created.

Cleaning Process: 

 Completed.!

--- Comparison ---
Organization:    12683  ->  12500  (Reduced: 183개)
Suborganization: 27670  ->  26615  (Reduced: 1055개)
